In [36]:
import icalendar
from datetime import datetime, timedelta

In [37]:
# Obtener la fecha actual
fecha_inicio = datetime(datetime.now().year, 2, 16)

# Obtener el último día del año actual
ultimo_dia_del_ano = datetime(fecha_inicio.year, 12, 31)

# Calcular los días restantes
dias_restantes = (ultimo_dia_del_ano - fecha_inicio).days

# Crear una lista de fechas desde la fecha actual hasta el último día del año
lista_fechas = []

for i in range(dias_restantes + 1):
    f = fecha_inicio + timedelta(days=i)
    g = f.replace(second=0, microsecond=0, minute=0, hour=0)
    
    if i % 6 == 0:  # Realizar la operación cada 4 iteraciones
        lista_fechas.append(g)  # Agregar la fecha a la lista

# Imprimir la lista de fechas
lista_fechas

[datetime.datetime(2024, 2, 16, 0, 0),
 datetime.datetime(2024, 2, 22, 0, 0),
 datetime.datetime(2024, 2, 28, 0, 0),
 datetime.datetime(2024, 3, 5, 0, 0),
 datetime.datetime(2024, 3, 11, 0, 0),
 datetime.datetime(2024, 3, 17, 0, 0),
 datetime.datetime(2024, 3, 23, 0, 0),
 datetime.datetime(2024, 3, 29, 0, 0),
 datetime.datetime(2024, 4, 4, 0, 0),
 datetime.datetime(2024, 4, 10, 0, 0),
 datetime.datetime(2024, 4, 16, 0, 0),
 datetime.datetime(2024, 4, 22, 0, 0),
 datetime.datetime(2024, 4, 28, 0, 0),
 datetime.datetime(2024, 5, 4, 0, 0),
 datetime.datetime(2024, 5, 10, 0, 0),
 datetime.datetime(2024, 5, 16, 0, 0),
 datetime.datetime(2024, 5, 22, 0, 0),
 datetime.datetime(2024, 5, 28, 0, 0),
 datetime.datetime(2024, 6, 3, 0, 0),
 datetime.datetime(2024, 6, 9, 0, 0),
 datetime.datetime(2024, 6, 15, 0, 0),
 datetime.datetime(2024, 6, 21, 0, 0),
 datetime.datetime(2024, 6, 27, 0, 0),
 datetime.datetime(2024, 7, 3, 0, 0),
 datetime.datetime(2024, 7, 9, 0, 0),
 datetime.datetime(2024, 7, 15, 

In [38]:
# Crear un nuevo objeto de calendario
calendario = icalendar.Calendar()

# Configurar las propiedades del evento
for day in lista_fechas:
    tempday = day
    for i in range(2):
        evento = icalendar.Event()
        evento.add('summary', 'trabajo_dia')
        evento.add('dtstart', tempday.replace(hour=7))
        evento.add('dtend', tempday.replace(hour=19))
        calendario.add_component(evento)
        tempday = tempday + timedelta(days=1)
        
    for i in range(2):
        evento = icalendar.Event()
        evento.add('summary', 'trabajo_noche')
        evento.add('dtstart', tempday.replace(hour=19))
        tempday = tempday + timedelta(days=1)
        evento.add('dtend', tempday.replace(hour=7))
        calendario.add_component(evento)
    
    evento = icalendar.Event()
    evento.add('summary', 'franco')
    evento.add('dtstart', tempday.replace(hour=7))
    tempday = tempday + timedelta(days=2)
    evento.add('dtend', tempday.replace(hour=7))
    calendario.add_component(evento)


# Guardar el calendario en un archivo .ics
with open('calendario.ics', 'wb') as f:
    f.write(calendario.to_ical())

print("Archivo 'evento.ics' creado con éxito.")

Archivo 'evento.ics' creado con éxito.


In [40]:
# Función para cargar eventos desde un archivo .ics
def cargar_eventos(archivo_ics):
    with open(archivo_ics, 'rb') as f:
        calendario = icalendar.Calendar.from_ical(f.read())
        eventos = []
        for evento in calendario.walk('vevent'):
            inicio = evento.get('dtstart').dt
            fin = evento.get('dtend').dt
            eventos.append((inicio, fin))
        return eventos

def encontrar_rangos(eventos, duracion_rango):
    eventos.sort(key=lambda x: x[0])  # Ordenar los eventos por su fecha de inicio
    rangos_sin_eventos = []

    for i in range(len(eventos)):
        inicio = eventos[i][0]
        fin = inicio + timedelta(days=duracion_rango - 1)  # Calcular la fecha final del rango

        # Verificar si el rango de días cumple con los criterios
        if fin < eventos[-1][1]:  # Si el final del rango está antes del último evento
            if all(("trabajo_dia" not in fecha or "trabajo_noche" not in fecha) and fecha[0].weekday() != 6 for fecha in eventos[i:i+duracion_rango]):
                rangos_sin_eventos.append((inicio, fin))

    return rangos_sin_eventos


# Ejemplo de uso
archivo_ics = 'calendario.ics'  # Cambia esto por la ruta de tu archivo .ics
eventos = cargar_eventos(archivo_ics)
rango = 12
rangos_sin_eventos = encontrar_rangos(eventos, rango)

for inicio, fin in rangos_sin_eventos:
    print("Rango sin eventos:", inicio, "-", fin)


Rango sin eventos: 2024-12-24 00:00:00 - 2025-01-04 00:00:00


In [39]:
def cargar_eventos(archivo_ics):
    with open(archivo_ics, 'rb') as f:
        calendario = icalendar.Calendar.from_ical(f.read())
        eventos = []
        for componente in calendario.walk():
            if componente.name == "VEVENT":
                inicio = componente.get("DTSTART")
                if isinstance(inicio, list):
                    inicio = inicio[0].dt
                else:
                    inicio = inicio.dt
                fin = componente.get("DTEND")
                if isinstance(fin, list):
                    fin = fin[0].dt
                else:
                    fin = fin.dt
                
                evento = {
                    "titulo": str(componente.get("SUMMARY")),
                    "inicio": inicio,
                    "fin": fin
                }
                eventos.append(evento)
        return eventos


# Ejemplo de uso
archivo_ics = 'calendario.ics'  # Reemplaza 'calendario.ics' con la ruta de tu archivo .ics
eventos = cargar_eventos(archivo_ics)

# Imprimir información de los eventos
for evento in eventos:
    print("Título:", evento["titulo"])
    print("Inicio:", evento["inicio"])
    print("Fin:", evento["fin"])
    print()

Título: trabajo_dia
Inicio: 2024-02-16 00:00:00
Fin: 2024-02-16 00:00:00

Título: trabajo_dia
Inicio: 2024-02-17 00:00:00
Fin: 2024-02-17 00:00:00

Título: trabajo_noche
Inicio: 2024-02-18 00:00:00
Fin: 2024-02-19 00:00:00

Título: trabajo_noche
Inicio: 2024-02-19 00:00:00
Fin: 2024-02-20 00:00:00

Título: franco
Inicio: 2024-02-20 00:00:00
Fin: 2024-02-22 00:00:00

Título: trabajo_dia
Inicio: 2024-02-22 00:00:00
Fin: 2024-02-22 00:00:00

Título: trabajo_dia
Inicio: 2024-02-23 00:00:00
Fin: 2024-02-23 00:00:00

Título: trabajo_noche
Inicio: 2024-02-24 00:00:00
Fin: 2024-02-25 00:00:00

Título: trabajo_noche
Inicio: 2024-02-25 00:00:00
Fin: 2024-02-26 00:00:00

Título: franco
Inicio: 2024-02-26 00:00:00
Fin: 2024-02-28 00:00:00

Título: trabajo_dia
Inicio: 2024-02-28 00:00:00
Fin: 2024-02-28 00:00:00

Título: trabajo_dia
Inicio: 2024-02-29 00:00:00
Fin: 2024-02-29 00:00:00

Título: trabajo_noche
Inicio: 2024-03-01 00:00:00
Fin: 2024-03-02 00:00:00

Título: trabajo_noche
Inicio: 2024-03-